This notebook will walk through the basics of spike detection and sorting using the raw data files recorded by the lab.

## Spike detection and sorting

We have two options. 

1) We can sort on our local computers, using WaveClus, a spikesorter that uses MATLAB. This is ok to do if you only have a couple of patients to sort. 

2) We can sort on the cluster. Preferred if you need to sort many subjects in parallel. 

## Option 1: Sort Locally on WaveClus

Download: https://github.com/csn-le/wave_clus/tree/2565067a2c16bee2bbbcbb25d81cf634d4ca04a1

## Option 2: Sort on the cluster using Combinato (or Python sorter of choice)

First, make sure you login to your Rhino account using the -XY flag to enable X11. You'll need this for a GUI later.

Also, it is possibly better not to install combinato dependencies in an environment. Instead, run it in the base environment? 

# Setup environment for use with Combinato spike sorter

1. ```conda create -n spike_clustering python=3.8 pyqt=5.9 numpy scipy xarray swig traits h5py matplotlib ipykernel qtpy```

2. ```source activate spike_clustering```

3. ```git clone https://github.com/jniediek/combinato.git```

4. ```pip install tables```

5. ```pip install PyWavelets```

6. ```cd ~/combinato```

7. ```python setup_options.py```

8. ```PATH=$PATH:/HOME/combinato```

9. ```PYTHONPATH=$PYTHONPATH:/HOME/```

10. ```LD_LIBRARY_PATH=$HOME/miniconda3/lib:$LD_LIBRARY_PATH```

11. ```export PATH PYTHONPATH LD_LIBRARY_PATH```

12. ```python tools/test_installation.py```

13. in /combinato/combinato/options.py, Line 46: ```folder_patterns``` should include ```’*chan*’```. This ensures that css-gui (you'll see this later) can actually find the results of the automatic spike sorting. 

14. in /combinato/combinato/extract/extract.py, Line 27: change 
```nWorkers = 5``` to ```nWorkers = 40```

15. in /combinato/combinato/extract/extract.py, Line 83: change 
```starts = list(range(0, size, 32000*5*60))``` to ```starts = list(range(0, size, 30000*1*10))```

16. in /combinato/combinato/extract/mp_extract.py, Line 113: change 
```sr = 32000.``` to ```sr = 30000.```

# Add it to the kernels for jupyter

1.  ```ipython kernel install --user --name=spike_clustering```

# Select this kernel for your notebook

# Download scripts to interface with Blackrock Recordings: 
https://www.blackrockmicro.com/wp-content/software/brPY.zip

# (optional) Download spikeinterface for alternatives (i.e. Klusta): 
1. ```pip install spikeinterface```
(see: https://spikeinterface.readthedocs.io/en/latest/installation.html)

2. ```pip install Cython tqdm click klusta klustakwik2```

## Where is the data? 

The data you are looking for is raw microwire data recorded at 30 KHz in .ns5 or .ns6 files. For example:

```/data10/RAM/subjects/R1278E/npt/20170224-145604_train_1/20170224-145604-001.ns6```

# ******** IMPORTANT NOTE ******** 

## Do NOT, under any circumstance, run your spike sorting in the "/data10/..." folders! It will overwrite the current spike-sorting results. 

## Copy the data to your own directory or to your local computer. If you are CONFIDENT that your spike-sorting is an improvement on the results in "/data10/..." then feel free to replace it... 


In [1]:
from os import path as ospath
import json

import h5py
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

In [ ]:
# Add the path for brPY library to jupyter path (replace with your own): 
import sys
brPY_path = '/home1/salman.qasim/Salman_Python_Scripts'
sys.path.append('{}/brPY'.format(brPY_path))
from brpylib import NsxFile

We record most of our single-neuron data using the BlackRock Microsystems Neuroport. This raw data is sampled at 30 kHz, meaning filesizes are very large, and is saved in proprietary .ns6 file formats. 

In [33]:
# Note that the directory is where I copied the data to. You should replace with your own. 
test_data = '/home1/salman.qasim/Estes_Bootcamp/Data/20170224-145604-001.ns6'

These data can't be read natively by most spike sorters. It's full of insane headers that have to be stripped first.

In [4]:
nsxfile = NsxFile(test_data)


20170224-145604-001.ns6 opened


In [5]:
cont_data = nsxfile.getdata()


Output data requested is larger than 1 GB, attempting to preallocate output now

First data packet requested begins at t = 0.003400 s, initial section padded with zeros
Data extraction requires paging: 1 of 4
Data extraction requires paging: 2 of 4
Data extraction requires paging: 3 of 4
Data extraction requires paging: 4 of 4


In [6]:
nsxfile.close()


20170224-145604-001.ns6 closed


In [7]:
cont_data.keys()
# Let's split a lot of this useful information into a params file and grab the data 

dict_keys(['elec_ids', 'start_time_s', 'data_time_s', 'downsample', 'data', 'data_headers', 'ExtendedHeaderIndices', 'samp_per_s'])

In [8]:
# Grab the ephys 
data = cont_data.pop('data')

In [32]:
# For Combinato (separate channels - save as matfiles)
## NOTE: CURRENTLY, Combinato hangs on large datasets. 

# Open a text file and write in header information 
path = ospath.splitext(nsxfile.datafile.name)[0]
with open('{}_params.txt'.format(path), 'w') as file:
     file.write(json.dumps(cont_data))
    
# Split the data by channels and write into hdf5 files
for ix, electrode in enumerate(cont_data['elec_ids']):
    if electrode != '129': # This is typically the sync pulse channel 
        ## HDF5: 
        f = h5py.File("{}_chan{}.hdf5".format(path, electrode), "w")
        f.create_dataset('data', data=data[ix, :], dtype='<i2')
        f.close()
        print('split electrode {}'.format(ix))
        #  Using the blackrock code
        #nsxfile.savesubsetnsx(elec_ids=[electrode], file_suffix='chan_{}'.format(electrodes))
        # Matfile: 
        #data_as_dict = {'data': data[ix, :11000000].astype('double'),
        #                 'sr':30000}
        #sio.savemat("{}_chan{}.mat".format(path, electrode), data_as_dict)


split electrode 0
split electrode 1
split electrode 2
split electrode 3
split electrode 4
split electrode 5
split electrode 6
split electrode 7
split electrode 8
split electrode 9
split electrode 10
split electrode 11
split electrode 12
split electrode 13
split electrode 14
split electrode 15
split electrode 16
split electrode 17
split electrode 18
split electrode 19
split electrode 20
split electrode 21
split electrode 22
split electrode 23
split electrode 24
split electrode 25
split electrode 26
split electrode 27
split electrode 28
split electrode 29
split electrode 30
split electrode 31


In [ ]:
# # For Klusta (all channels together, no splitting - HDF5)

# # Grab the ephys 
# data = cont_data.pop('data')
# # Open a text file and write in header information 
# path = ospath.splitext(nsxfile.datafile.name)[0]
# with open('{}_params.txt'.format(path), 'w') as file:
#      file.write(json.dumps(cont_data))
    
# # save the data to hdf5 
# f = h5py.File("{}.hdf5".format(path), "w")
# f.create_dataset('dataset', data=data, dtype='<i2')
# f.close()


In [36]:
# Write a bash script to run the automatic spike sorting for all the data 
# Note, because Combinato currently doesn't support parallel processing for non-Neuralynx data, we use css-simple-clustering. 

path_to_sh = '/home1/salman.qasim/Estes_Bootcamp'
path_to_split_files= '/home1/salman.qasim/Estes_Bootcamp/Data'
path_to_save_jobfiles= '/home1/salman.qasim/Estes_Bootcamp/Combinato_clustering'
user_name = path_to_sh.split('/')[2][0:3]


# Extract and cluster NEGATIVE polarity spikes 
with open ('{}/run_combinato_neg.sh'.format(path_to_sh), 'w') as bash_script:
        bash_script.write('''
                    #!/bin/bash
                    declare -a arr=({path_to_split_files})
                    mkdir {path_to_save_jobfiles}
                    mkdir {path_to_save_jobfiles}/$(date '+%d-%b-%Y')
                    cd {path_to_save_jobfiles}/$(date '+%d-%b-%Y')
                    touch do_manual_neg.txt
                    for file in "${{arr[@]}}"; do
                        cd $file
                        for chan in *.mat; do
                            css-extract --files ${{chan%%.*}}.hdf5 --h5
                        css-mask-artifacts
                        ls */*.h5 > do_sort.txt
                        css-prepare-sorting --neg --jobs do_sort.txt
                        css-cluster --jobs do_sort.txt
                        css-combine --jobs do_sort.txt
                        done
                    done
                '''.format(path_to_split_files=path_to_split_files,
                           path_to_save_jobfiles=path_to_save_jobfiles,
                           user_name=user_name))


There's probably a way to run this bash script well from your notebook. But I couldn't figure that out (see commented code below).

In [109]:
# import subprocess
# subprocess.call(['{}/run_combinato.sh'.format(path_to_sh)], shell=True)

So, head to your terminal. 

Make sure you type in

``` source activate spike_clustering```

to ensure you're in the proper environment. Then, 

```cd /home1/salman.qasim/Estes_Bootcamp/Data```

``` bash run_combinato_neg.sh```

This might take some time. This is the automatic clustering, without human input. 

In [ ]:
# cluster POSITIVE polarity spikes 
with open ('{}/run_combinato_pos.sh'.format(path_to_sh), 'w') as bash_script:
        bash_script.write('''
                    #!/bin/bash
                    declare -a arr=({path_to_split_files})
                    mkdir {path_to_save_jobfiles}
                    mkdir {path_to_save_jobfiles}/$(date '+%d-%b-%Y')
                    cd {path_to_save_jobfiles}/$(date '+%d-%b-%Y')
                    touch do_manual_neg.txt
                    for file in "${{arr[@]}}"; do
                        cd $file
                        ls */*.h5 > do_sort.txt
                        css-prepare-sorting --jobs do_sort.txt
                        css-cluster --jobs do_sort.txt
                        css-combine --jobs do_sort.txt
                        done
                    done
                '''.format(path_to_split_files=path_to_split_files,
                           path_to_save_jobfiles=path_to_save_jobfiles,
                           user_name=user_name))

So, head to your terminal. 

Make sure you type in

``` source activate spike_clustering```

to ensure you're in the proper environment. Then, 

```cd /home1/salman.qasim/Estes_Bootcamp/Data```

``` bash run_combinato_pos.sh```

This might take some time. This is the automatic clustering, without human input. 

Once this is done, there should be a lot of new folders in "/home1/salman.qasim/Estes_Bootcamp/Data".

Before you can use the GUI to manually sort through these results, you HAVE TO DEACTIVATE your environment. This is dumb! But otherwise Python can't find the plugin for displaying the GUI, which is not environment specific. 

``` source deactivate spike_clustering```

```cd /home1/salman.qasim/Estes_Bootcamp/Data```

```css-gui```

This should bring up the GUI for manual sorting. If you did step 13 of the Combinato setup correctly, you should be able to see the different channel folders when you select "Open" from the drop-down menu.

## Everything below here is SpikeInterface test code that I haven't quite figured out.

In [112]:
# # Spikeinterfaces example using klusta
# import spikeinterface.sorters as ss
# import spikeinterface.toolkit as st
# f = h5py.File('/home1/salman.qasim/Estes_Bootcamp/Data/20170224-145604-001_chan101.hdf5', 'r')
# timeseries =  f['data'].value
# timeseries = timeseries.reshape([1, len(timeseries)])
# sampling_frequency = 30000
# recording = se.NumpyRecordingExtractor(timeseries=timeseries, geom=None, sampling_frequency=sampling_frequency)


In [111]:
# sorting_KL = ss.run_klusta(recording, output_folder='/home1/salman.qasim/Estes_Bootcamp/Data/tmp_KL', verbose=True)
# print('Units found with Klusta:', sorting_KL.get_unit_ids())

In [113]:
# templates = st.postprocessing.get_unit_templates(recording, sorting_KL, max_spikes_per_unit=200,
#                                                  save_as_property=True, verbose=True)

In [114]:
# firing_rates = st.validation.compute_firing_rates(sorting_KL, duration_in_frames=recording.get_num_frames())
# isi_violations = st.validation.compute_isi_violations(sorting_KL, duration_in_frames=recording.get_num_frames(), isi_threshold=0.0015)
# snrs = st.validation.compute_snrs(recording=recording, sorting=sorting_KL, max_spikes_per_unit_for_snr=1000)
# nn_hit_rate, nn_miss_rate = st.validation.compute_nn_metrics(recording=recording, sorting=sorting_KL, num_channels_to_compare=13)

In [118]:
# st.postprocessing.export_to_phy(recording, sorting_KL, output_folder='phy', verbose=True)
# # Or from a notebook:  !phy template-gui phy/params.py
# # After manual curation you can load back the curated data using the PhySortingExtractor: